# Check flat fielded images from NGDEEP

This is to look at the images and see if there are problems.

In [ ]:
from jdaviz import Imviz
import warnings
from astropy.io import ascii, fits
from astropy.table import Table
from astropy.coordinates import SkyCoord, Angle
import astropy.units as u
import numpy as np
import os
import pandas as pd
import glob

## Read list of files

In [ ]:
def write_keywords_csv(datadir):
    # set up an empty dataframe initially to fill in
    df = pd.DataFrame(columns=["FILENAME",
                               "TARG_RA", 
                               "TARG_DEC", 
                               "FILTER", # Grism; GR150R/GR150C
                               "PUPIL", # Filter used; F090W, F115W, F140M, F150W F158M, F200W
                               "PATT_NUM", # Position number within dither pattern for WFSS
                               "NUMDTHPT", # Total number of points in entire dither pattern
                               "XOFFSET", # X offset from pattern starting position for NIRISS (arcsec)
                               "YOFFSET"]) # Y offset from pattern starting position for NIRISS (arcsec)

    for ratefile in glob.glob(os.path.join(datadir, "*rate.fits")):
        image = fits.open(ratefile)

        # create a new dataframe for each file
        df2 = pd.DataFrame({"FILENAME"   : ratefile,
                            "TARG_RA"     : [image[0].header["TARG_RA"]],
                            "TARG_DEC"     : [image[0].header["TARG_DEC"]],
                            "FILTER"     : [image[0].header["FILTER"]],
                            "PUPIL"      : [image[0].header["PUPIL"]],
                            "PATT_NUM"   : [image[0].header["PATT_NUM"]],
                            "NUMDTHPT"   : [image[0].header["NUMDTHPT"]],
                            "XOFFSET"    : [image[0].header["XOFFSET"]],
                            "YOFFSET"    : [image[0].header["YOFFSET"]]
                           })

        # merge the two dataframes together to create a dataframe with all 
        df = pd.concat([df, df2], ignore_index=True, axis=0)

    dfsort = df.sort_values('FILENAME', ignore_index=False)

    dfsort.to_csv("./list_ngdeep_rate.csv", sep=',', index=False)
    return dfsort

In [ ]:
listrate_file = './list_ngdeep_rate.csv'

if os.path.exists(listrate_file):
    rate_df = Table.read(listrate_file)
else:
    rate_df = write_keywords_csv('data/NGDEEP/rate')

rate_df[-1::]

While you could look at the rate images, instead consider running the files through flatfield step of the pipeline to clean up detector artifacts.

** Insert notebook 02 here as an optional function and call it. Fix the function to use call instead while doing this, too ** 

## Check astrometry on direct images

In [ ]:
direct_image_df = rate_df[rate_df['FILTER'] == 'CLEAR'] # selecting out only the direct images
clearim = direct_image_df['FILENAME']
# optional flatfield step would be:
flatfield_im = [f.replace('rate.fits', 'flatfieldstep.fits') for f in clearim]
datalabel = direct_image_df['PUPIL']

### Use Imviz batch load since it is many images
It takes about 1 minute to load all the images.

** I need some basic instructions here to know how to navigate **

In [ ]:
imviz = Imviz()
for img, filt_label in zip(clearim, datalabel):
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        imviz.load_data(img, data_label=filt_label)
imviz.show()

In [ ]:
viewer = imviz.default_viewer

In [ ]:
linking = imviz.plugins['Links Control']
linking.link_type = 'WCS'

In [ ]:
# This also takes a bit of time (~30 seconds) to loop through all the images
plotopt = imviz.plugins['Plot Options']
plotopt.multiselect = True
plotopt.select_all(viewers=True, layers=True)
plotopt.stretch_preset = '99.5%'
plotopt.multiselect = False
#plotopt.stretch_function = 'Arcsinh'
#plotopt.stretch_vmin = -0.002
#plotopt.stretch_vmax = 0.16

### Blink
Using the "b" key when on the viewer.

**Data note**: there is a problem with the astrometry. The F200W filter does not match the others. I am not sure if F200W is the problem or the other two are the problem. Fixing the astrometry is out of scope for this notebook.

## Check direct images and their corresponding WFSS images

Look at direct and dispersed images on the same dither position.

In [ ]:
# Using the XOFFSET and YOFFSET keywords, match the direct image and dispersed (grism) images to each other

file_dict = {}
for xoff, yoff, imgname in zip(direct_image_df['XOFFSET'], direct_image_df['YOFFSET'], 
                               direct_image_df['FILENAME']):
    match_pointing_df = rate_df[(rate_df['XOFFSET'] == xoff) & 
                                (rate_df['YOFFSET'] == yoff) &
                                (rate_df['FILENAME'] != imgname)]
    
    # If there aren't any matches, keep moving
    if not len(match_pointing_df):
        continue
    
    # fill in the direct / dispersed dataframe for each of the filters
    temp_filter = match_pointing_df.iloc[0]['PUPIL']
    if temp_filter in list(file_dict.keys()):
        # For now, only grab the first instance of the image/grism pair
        continue
    else:
        print('**', temp_filter, imgname)
        print(match_pointing_df.iloc[0]['FILENAME'])
        print(match_pointing_df.iloc[0]['FILTER'])
        file_dict[temp_filter] = {'direct_img' : imgname,
                                  'dispersed_img' : match_pointing_df.iloc[0]['FILENAME']}
    print()

In [ ]:
imviz2 = Imviz()
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    for filt, img_dict in file_dict.items(): # loop over all filters
        for imgtype, filename in img_dict.items(): # loop over all files
            imviz2.load_data(filename, data_label=f"{filt} {imgtype}")
imviz2.show()

#### Manual Check

In [ ]:
filedir = 'data/NGDEEP/rate/'
dir_f115w = filedir + 'jw02079004001_04101_00004_nis_flatfieldstep.fits'
disp_f115w = filedir + 'jw02079004001_05101_00001_nis_flatfieldstep.fits'

dir_f150w = filedir + 'jw02079004002_10101_00004_nis_flatfieldstep.fits'
disp_f150w = filedir + 'jw02079004002_11101_00001_nis_flatfieldstep.fits'

dir_f200w = filedir + 'jw02079004003_04101_00004_nis_flatfieldstep.fits'
disp_f200w = filedir + 'jw02079004003_05101_00001_nis_flatfieldstep.fits'

In [ ]:
imviz2 = Imviz()
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    imviz2.load_data(dir_f115w, data_label='F115W dir')
    imviz2.load_data(disp_f115w, data_label='F115W wfss')
    imviz2.load_data(dir_f150w, data_label='F150W dir')
    imviz2.load_data(disp_f150w, data_label='F150W wfss')
    imviz2.load_data(dir_f200w, data_label='F200W dir')
    imviz2.load_data(disp_f200w, data_label='F200W wfss')
imviz2.show()

Split direct images and dispersed images in two separate viewers.

In [ ]:
plotopt = imviz2.plugins['Plot Options']
plotopt.multiselect = True
plotopt.select_all(viewers=True, layers=True)
plotopt.stretch_preset = '99.5%'
plotopt.multiselect = False